In [6]:
import csv
import sys
import numpy as np

In [7]:
te_list = ['copia',
          'MAGGY',
          'PYRET',
          'GYMAG1',
          'GYMAG2',
          'MGRL3']
eccs_file_list = ['lengthfiltered.merged.splitreads.G3_1A.bed',
                 'lengthfiltered.merged.splitreads.G3_1B.bed',
                 'lengthfiltered.merged.splitreads.G3_1C.bed',
                 'lengthfiltered.merged.splitreads.G3_2A.bed',
                 'lengthfiltered.merged.splitreads.G3_2B.bed',
                 'lengthfiltered.merged.splitreads.G3_2C.bed',
                 'lengthfiltered.merged.splitreads.G3_3A.bed',
                 'lengthfiltered.merged.splitreads.G3_3B.bed']

In [12]:
te_dict = {}

for te in te_list:
    te_file = 'tmp.'+te+'.G3_1A.filtered.bed'
    ltr_list = [[] for i in range(56)]
    with open(te_file, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            if 'LTR' in row[3]:
                ltr_list[int(row[0][10:12])-1].append([int(row[1]), int(row[2])])
    internal_list = [[] for i in range(56)]
    with open(te_file, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            if 'INTERNAL' in row[3]:
                internal_list[int(row[0][10:12])-1].append([int(row[1]), int(row[2])])
    ltr_list_arrays = []
    for i in range(len(ltr_list)):
        ltr_list_arrays.append(np.array(ltr_list[i], dtype=object))  
    internal_list_arrays = []
    for i in range(len(internal_list)):
        internal_list_arrays.append(np.array(internal_list[i], dtype=object))
    te_dict[te+'LTR'] = ltr_list_arrays
    te_dict[te+'INTERNAL'] = internal_list_arrays

eccs_list = [[] for i in range(56)]
for eccs_file in eccs_file_list:
    with open(eccs_file, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            eccs_list[int(row[0][10:12])-1].append([int(row[1]), int(row[2])])
            
eccs_list_arrays = []
for i in range(len(eccs_list)):
    eccs_list_arrays.append(np.array(eccs_list[i], dtype=object))

In [13]:
te_dict['MAGGYINTERNAL']

[array([[2162815, 2167967],
        [2344235, 2349366],
        [2377068, 2382199],
        [2383386, 2388516],
        [2389705, 2394836],
        [2562523, 2567654],
        [3224925, 3228426],
        [4645331, 4650484],
        [5358713, 5360148],
        [5360402, 5365533],
        [5365787, 5368222]], dtype=object),
 array([[3951, 6245],
        [148909, 154040],
        [158777, 163908],
        [165097, 170228],
        [227560, 232691],
        [240677, 245829],
        [511526, 512352],
        [515222, 520353],
        [533251, 537560],
        [1159053, 1164205],
        [3855078, 3860209],
        [3861424, 3866576],
        [4117071, 4122202],
        [4123433, 4128562],
        [4134437, 4139567],
        [4175091, 4180222],
        [4189422, 4194553],
        [4272442, 4276511],
        [4276494, 4277457],
        [4992810, 4997962]], dtype=object),
 array([[4, 3934],
        [24211, 29342],
        [30518, 35649],
        [878594, 883725],
        [897797, 900450],
   

In [ ]:
# "MQOP010000"+str(i+1).zfill(2)+".1"

In [39]:
eccs_double_ltr_overlaps = []

for i in range(len(eccs_list_arrays)):
    scaffold_eccs = eccs_list_arrays[i]
    scaffold_ltrs = ltr_list_arrays[i]
    if np.shape(scaffold_ltrs)[0] > 0:
        for ecc in scaffold_eccs:
            ltr_overlaps = scaffold_ltrs[np.logical_or(
                                np.logical_and(ecc[0] > scaffold_ltrs[:,0], ecc[0] < scaffold_ltrs[:,1]),
                                np.logical_and(ecc[1] > scaffold_ltrs[:,0], ecc[1] < scaffold_ltrs[:,1])
                                                      )]
            if np.shape(ltr_overlaps)[0] >= 2:
                eccs_double_ltr_overlaps.append([i, ecc[0], ecc[1]])

In [27]:
print(eccs_double_ltr_overlaps)

[[3, 713990, 714137], [3, 4036144, 4036273], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036085, 4036220], [3, 4036085, 4036220], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036142, 4036274], [3, 4036136, 4036299], [4, 362509, 362810], [4, 439540, 439704], [4, 439540, 439704], [6, 403978, 404147], [6, 403978, 404147], [13, 600536, 604569], [13, 595125, 597597], [13, 595125, 597597], [13, 595125, 597597], [13, 635090, 635306], [13, 635090, 635306], [13, 595125, 597597], [15, 61026, 61358], [15, 61026, 61358], [15, 61023, 61360], [15, 61042, 61341], [15, 60995, 61388], [15, 60995, 61388], [16, 372979, 373769], [16, 372978, 373778], [27, 63540, 65848]]


In [41]:
eccs_double_ltr_overlaps_w_internal = []

for ecc in eccs_double_ltr_overlaps:
    scaffold_internals = internal_list_arrays[ecc[0]]
    if np.shape(scaffold_internals)[0] > 0:
        internal_overlaps = scaffold_internals[np.logical_and(
            ecc[1] < scaffold_internals[:,0],
            ecc[2] > scaffold_internals[:,1]
        )]
        if np.shape(internal_overlaps)[0] > 0:
            eccs_double_ltr_overlaps_w_internal.append(["MQOP010000"+str(ecc[0]+1).zfill(2)+".1", ecc[1], ecc[2]])

In [42]:
print(eccs_double_ltr_overlaps_w_internal)

[['MQOP01000003.1', 1871568, 1873843], ['MQOP01000003.1', 1871568, 1873843], ['MQOP01000003.1', 1871568, 1873843], ['MQOP01000003.1', 1871568, 1873843], ['MQOP01000003.1', 1871568, 1873843], ['MQOP01000004.1', 4036166, 4039881], ['MQOP01000004.1', 4036166, 4039881], ['MQOP01000007.1', 403978, 404147], ['MQOP01000007.1', 403978, 404147], ['MQOP01000007.1', 403971, 404183], ['MQOP01000007.1', 403971, 404183], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000007.1', 403976, 404174], ['MQOP01000010.1', 196481, 197211], ['MQOP01000010.1', 196497, 197194], ['MQOP01000010.1', 196488, 197207], ['MQOP01000010.1', 196488, 197207], ['MQOP01000011.1', 469775, 477382], ['MQOP01000011.1', 469775, 477382], ['MQOP01000011.1', 469775, 477382], ['MQOP01000011.1', 469775, 477382], ['MQOP01000011.1', 469775, 477382], ['MQOP0100001

In [10]:
with open('eccs_double_ltr_overlaps.bed', 'w', newline = '') as out:
    w = csv.writer(out, delimiter ='\t')
    for row in eccs_double_ltr_overlaps:
        w.writerow(row)

In [43]:
with open('eccs_double_ltr_overlaps_w_internal.bed', 'w', newline = '') as out:
    w = csv.writer(out, delimiter ='\t')
    for row in eccs_double_ltr_overlaps_w_internal:
        w.writerow(row)